In [20]:
pip install datasets transformers torch unsloth

In [21]:
from datasets import load_dataset

dataset = load_dataset("ai4privacy/pii-masking-65k")
train_data = dataset["train"]

README.md:   0%|          | 0.00/6.16k [00:00<?, ?B/s]

english_balanced_10k.jsonl:   0%|          | 0.00/14.1M [00:00<?, ?B/s]

french_balanced_5k.jsonl:   0%|          | 0.00/8.31M [00:00<?, ?B/s]

german_balanced_3k.jsonl:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

italian_balanced_3k.jsonl:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21588 [00:00<?, ? examples/s]

In [22]:
print(train_data.column_names)

['masked_text', 'unmasked_text', 'token_entity_labels', 'tokenised_unmasked_text']


In [24]:
def format_data(example):
    return {
        "input": example["unmasked_text"],
        "output": example["masked_text"]
    }

formatted_data = train_data.map(format_data)

Map:   0%|          | 0/21588 [00:00<?, ? examples/s]

In [26]:
def format_chat(example):
    messages = [
        {"role": "system", "content": "You are a privacy assistant. Redact all personally identifiable information (PII) from the following text."},
        {"role": "user", "content": example["unmasked_text"]},
        {"role": "assistant", "content": example["masked_text"]}
    ]
    return {"messages": messages}

formatted_chat_data = train_data.map(format_chat)

Map:   0%|          | 0/21588 [00:00<?, ? examples/s]

In [27]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)  # Should match your CUDA version

True
12.6


In [28]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct", # "unsloth/Phi-3-mini-4k-instruct loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.5.9: Fast Mistral patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [29]:
from unsloth.chat_templates import get_chat_template

In [30]:
def tokenize_function(example):
    # tokenizer.apply_chat_template returns a tensor if return_tensors="pt"
    # We need to return a dict with a key like "input_ids"
    input_ids = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=True,
        add_generation_prompt=False,
        return_tensors="pt"
    )
    # Convert to list so it can be stored in the dataset
    return {"input_ids": input_ids[0].tolist()}

dataset_tokenized = formatted_chat_data.map(tokenize_function)

Map:   0%|          | 0/21588 [00:00<?, ? examples/s]

In [31]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['masked_text', 'unmasked_text', 'token_entity_labels', 'tokenised_unmasked_text'],
        num_rows: 21588
    })
})


In [32]:
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"CUDA: {torch.version.cuda}")
print(f"BF16 support: {torch.cuda.is_bf16_supported()}")

GPU: Tesla T4
CUDA: 12.6
BF16 support: False


In [33]:
import os
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Environment fSetup for T4 GPUs
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Helps with debugging

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenization with dynamic padding
dataset_tokenized = dataset.map(
    lambda x: tokenizer(
        x["masked_text"],
        truncation=True,
        max_length=1024,
        return_tensors=None,
    ),
    batched=True,
    remove_columns=dataset["train"].column_names
)

# LoRA configuration remains the same
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["qkv_proj", "o_proj", "down_proj", "up_proj", "gate_proj"],
    lora_alpha=16,
    lora_dropout=0,
)

# Updated training arguments for T4 compatibility
training_args = TrainingArguments(
    output_dir="./results",
    report_to="none",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    fp16=True,  # Use fp16 instead of bf16 for T4 compatibility
    learning_rate=2e-5,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=1000,
    optim="adamw_torch",
    gradient_checkpointing=True,
    torch_compile=False,  # Disable torch.compile for stability
)

# Data collator remains unchanged
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_tokenized["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    preprocess_logits_for_metrics=lambda logits, labels: logits
)

# Start training
trainer.train()

Map:   0%|          | 0/21588 [00:00<?, ? examples/s]

Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!


<ipython-input-33-bec9382a6fed>:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 21,588 | Num Epochs = 1 | Total steps = 5,397
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 20,447,232/4,000,000,000 (0.51% trained)


Step,Training Loss
100,2.485800
200,2.008700
300,1.706400
400,1.576200
500,1.601100
600,1.562900
700,1.545900
800,1.546100
900,1.496400
1000,1.514100


TrainOutput(global_step=5397, training_loss=1.472142670316698, metrics={'train_runtime': 5339.4651, 'train_samples_per_second': 4.043, 'train_steps_per_second': 1.011, 'total_flos': 4.22059738858537e+16, 'train_loss': 1.472142670316698, 'epoch': 1.0})

In [34]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [35]:
FastLanguageModel.for_inference(model)
messages = [
    {"role": "system", "content": "You are a privacy assistant. Redact all personally identifiable information (PII) from the following text."},
    {"role": "user", "content": "My name is Abhishek Sharma and my email is something.something@example.com."}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")
outputs = model.generate(inputs, max_new_tokens=128)
print(tokenizer.batch_decode(outputs)[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|system|> You are a privacy assistant. Redact all personally identifiable information (PII) from the following text.<|end|><|user|> My name is Abhishek Sharma and my email is something.something@example.com.<|end|><|assistant|> [FIRSTNAME_1] [LASTNAME_1], we have noticed that your email address [EMAIL_1] has been shared in a public forum. As a privacy assistant, we advise you to change your email address to [EMAIL_2] to ensure your privacy.<|end|>
